<a href="https://colab.research.google.com/github/SidCodes0001/RAG-Document-Processing-Bot/blob/main/RAG_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain
!pip -q install transformers
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers
!pip install chromadb
!pip install openai
!pip install tiktoken


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from transformers import HuggingFacePipeline



from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains import RetrievalQA


from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI


import os
import sys





## Load the Documents and Extract Text From Them

In [ ]:
!mkdir docs


upload your files in this docs folder

In [ ]:

document=[]
for file in os.listdir("docs"):
  if file.endswith(".pdf"):
    pdf_path="./docs/"+file
    loader=PyPDFLoader(pdf_path)
    document.extend(loader.load())
  elif file.endswith('.docx') or file.endswith('.doc'):
    doc_path="./docs/"+file
    loader=Docx2txtLoader(doc_path)
    document.extend(loader.load())
  elif file.endswith('.txt'):
    text_path="./docs/"+file
    loader=TextLoader(text_path)
    document.extend(loader.load())

In [ ]:
document


In [ ]:
len(document)


## Split the Document into Chunks

- character text splitter
- RecursiveCharacterTextSplitter

In [ ]:
document_splitter=CharacterTextSplitter(separator='\n', chunk_size=500, chunk_overlap=100)

document_chunks=document_splitter.split_documents(document)

document_chunks[0]



## Download the Embeddings from Hugging Face, Download the Sentence Transformer Embeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

embeddings = HuggingFaceEmbeddings()



In [ ]:
# os.environ["OPENAI_API_KEY"] = ""
# embeddings = OpenAIEmbeddings()
# embeddings


## Setting Up Chroma as our Vector Database

In [ ]:
vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')
vectordb.persist()



## FAISS AS VECTOR DB

In [ ]:
# vectorstore=FAISS.from_documents(text_chunks, embeddings)


In [ ]:
notebook_login()


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_4bit=True,
                                            #load_in_8bit=True,
                                            # load_in_16bit=True
                                             )

## Creating a Hugging Face Pipeline

In [ ]:

pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              torch_dtype=torch.bfloat16,
              device_map='auto',
              max_new_tokens=512,
              min_new_tokens=-1,
              top_k=30,
              eos_token_id=tokenizer.eos_token_id
              )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


for open AI llm

In [ ]:
# llm=ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')


## Creating a memory object which is necessary to track inputs/outputs and hold a conversation

In [ ]:
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)


## Creating a Conversation Retrieval QA Chain

- The ConversationalRetrievalQA chain builds on RetrievalQAChain to provide a chat history component.



In [ ]:
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                             retriever=vectordb.as_retriever(search_kwargs={'k':6}),
                                             verbose=False, memory=memory)

- RETRIEVAL QA SOURCE CHAIN

In [ ]:
# chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())
# result=chain({"question": "How good is Vicuna?"}, return_only_outputs=True)


In [ ]:
result=pdf_qa({"question":"YOLOv7 is trained on which dataset"})
result['answer']
